## 04. Извлечение вопрос-ответных пар из текста

Для тренировки Yandex GPT нам необходимо сформировать множество вопрос-ответных пар. Чтобы сделать это на основе текста, мы попробуем использовать большие языковые модели.

Для начала, поэкспериментируем с Yandex GPT:

In [2]:
import json
from langchain.llms import YandexGPT

config = json.load(open("config.json"))

gpt = YandexGPT(api_key=config['api_key'])

Для извлечения вопрос-ответных пар поэкспериментируем с разными запросами. Помните про основные правила промпт-инжиниринга:

* Четкое выделение входных данных
* Подробные инструкции
* Указание желаемого формата данных на выходе
* Экспериментируйте с разными вариантами промптов
* Экспериментируйте с температурой

In [7]:
prompt="""
Ниже в тройных обратных кавычках приводится некоторый текст.
Твоя задача - внимательно прочитать
текст и сформулировать как можно больше (но не менее 10) вопросов к 
этому тексту. Пожалуйста, запиши вопросы и ответы на них (содержащиеся 
в тексте) в формате JSON следующим образом:
[
    {"request":"вопрос 1","response":"ответ 1"},
    {"request":"вопрос 2","response":"ответ 2"},
    ...
].
Напиши только те вопросы, ответы на которые содержатся в тексте. 
Пожалуйста, формулируй вопросы и ответы как можно более подробным образом.
Постарайся, чтобы в вопросе содержалась необходимая информация об объекта, 
например, не спрашивай "как произносится заклинание?", а пиши подробно:
"как произносится заклинание невидимости".
Вот фрагмент странички:
```
{PAGE}
```
"""

Используйте код ниже, чтобы извлекать вопрос-ответные пары из страничек и сразу смотреть результаты. Мы используем класс `SimpleJsonOutputParser` из LangChain, чтобы лишний раз убедиться, что результат LLM представлен в формате JSON.

Скорее всего, вы обнаружите следующее:
* Некоторые странички слишком короткие, их нет смысла обрабатывать. Можно обрабатывать странички по частям, например, сначала брать все заклинания (маска файлов `content/text/Закли*`), потом - книги/фильмы (маска файлов `content/text/Гарри Поттер и*`) и т.д.
* Один промпт не подходит на все случаи жизни. Для каждого типа страниц может помочь слегка модифицировать промпт.
* Некоторые вопросы могут иметь вид: `{'request': 'Как называется заклинание?', 'response': 'Заклинание воспламенения'}` - они не очень подходят для тренировки модели, поскольку в вопросе не упоминается требуемое заклинание. Нужно попытаться как-то так сформулировать промпт, чтобы вопросы были как можно более специфическими.

Все вопрос-ответные пары мы запоминаем в большом словаре `pairs`. Можно инициализировать словарь только один раз, и затем уже пополнять его, запуская несколько раз цикл обхода страничек:

In [10]:
pairs = {}

In [16]:
import os,time
from glob import glob
from grpc._channel import _MultiThreadedRendezvous

from langchain.output_parsers.json import SimpleJsonOutputParser

jp = SimpleJsonOutputParser()
gpt.temperature = 0.1

def process_files(fns):
  for fn in fns:
    if fn in pairs.keys():
        print(f"Already processed: {fn}")
        continue
    print(f'Processing {fn}')
    text = open(fn,'r',encoding='utf-8').read()
    p = prompt.replace('{PAGE}',text)
    try:
        res = gpt(p)
    except _MultiThreadedRendezvous as e:
        print(f"Error processing {fn}")
        continue
    j = jp.parse(res)
    print(j)
    if j is not None and len(j)>0:
        pairs[fn] = j
    time.sleep(1)

process_files(glob('content/text/Закли*'))

Processing content/text\Заклинание «Сито из котла».txt
[]
Already processed: content/text\Заклинание анимагии.txt
Already processed: content/text\Заклинание вечного приклеивания.txt
Already processed: content/text\Заклинание воспламенения.txt
Already processed: content/text\Заклинание гидрокинеза.txt
Already processed: content/text\Заклинание головного пузыря.txt
Already processed: content/text\Заклинание для мытья посуды.txt
Already processed: content/text\Заклинание для окрашивания Коросты в жёлтый цвет.txt
Already processed: content/text\Заклинание для очистки картофеля.txt
Already processed: content/text\Заклинание для устранения заклинившего реверса.txt
Already processed: content/text\Заклинание заметания следов.txt
Already processed: content/text\Заклинание Заморозки.txt
Processing content/text\Заклинание запечатывания комнаты.txt
[]
Processing content/text\Заклинание запечатывания пергамента.txt
[]
Already processed: content/text\Заклинание зонта.txt
Already processed: content/t

Также есть несколько страниц, по которым важно построить вопрос-ответные пары:

In [19]:
prompt="""
Ниже в тройных обратных кавычках приводится список разных объектов.
Твоя задача - внимательно прочитать
текст и сформулировать как можно больше (но не менее 10) вопросов к 
этому тексту. Пожалуйста, запиши вопросы и ответы на них (содержащиеся 
в тексте) в формате JSON следующим образом:
[
    {"request":"вопрос 1","response":"ответ 1"},
    {"request":"вопрос 2","response":"ответ 2"},
    ...
].
Напиши только те вопросы, ответы на которые содержатся в тексте. 
Пожалуйста, формулируй вопросы и ответы как можно более подробным образом.
Постарайся, чтобы в вопросе содержалась необходимая информация об объекте, 
например, не спрашивай "как произносится заклинание?", а пиши подробно:
"как произносится заклинание невидимости".
Вот список:
```
{PAGE}
```
"""

cool_pages = [
    'Великие волшебники', 'Список волшебных палочек', 'Список заклинаний',
    'Список зелий', 'Список волшебных существ Саламандера'
]

process_files([f'content/text/{x}.txt' for x in cool_pages])

Processing content/text/Великие волшебники.txt
Error processing content/text/Великие волшебники.txt
Already processed: content/text/Список волшебных палочек.txt
Processing content/text/Список заклинаний.txt
None
Processing content/text/Список зелий.txt
[{'request': 'какие зелья упоминаются в книгах о Гарри Поттере?', 'response': 'Амортенция, антисглазной лак, бедовый лосьон, бодроперцовое зелье, болтливое зелье, болтушка для молчунов, вечные прочные ресницы, волчье противоядие, волшебный искристый порошок, всевозможно зелье, гарантированный десятисекундный прыщевыводитель, гербицид, глазная сыворотка «Любовь слепа», доксицид, долголетние эликсиры, драконий тоник, дурманящая настойка, душильный газ, дыбоволосное зелье, елейная смазка Григория, желчь броненосца, животворящий эликсир, зелье взрыва, зелье для излечения фурункулов, зелье для проявки фотографий, зелье забывчивости, зелье красоты, зелье Мопсуса, зелье невидимости, зелье необычайной силы, зелье от икоты, зелье от кашля, зелье 

Запишем на всякий случай получившийся список pairs:

In [17]:
with open('data/pairs.json','w',encoding='utf-8') as f:
    json.dump(pairs,f,ensure_ascii=False)

Теперь преобразуем всё к формату, в котором Yandex GPT принимает фразы для до-обучения:

In [21]:
res = []
stop = ['как называется заклинание?','как произносится заклинание?']
for _,p in pairs.items():
    for d in p:
        if 'request' in d and 'response' in d and not(d['request'].lower() in stop):
            res.append(d)
            
print(f"Всего {len(res)} вопрос-ответных пар")
            

Всего 720 вопрос-ответных пар


Запишем результат на диск:

In [22]:
with open('data/pairs_for_train_ygpt.json','w',encoding='utf-8') as f:
    json.dump(res,f,ensure_ascii=False)

### Пробуем диалоговые LLM

Для получения дополнительного списка вопрос-ответных пар можно попробовать использовать диалоговые модели с длинным контекстом, например, [Claude 2](https://claude.ai) (*не забудьте про [VPN](https://veepn.com/home/)!**). Для этого сформируем несколько файлов с текстом по определённым темам, которые мы затем загрузим в модель вручную:

In [23]:
page_classes = {
    "Заклинания" : "Заклинани*",
    "Практикумы" : "Практикум*",
    "Портреты" : "Портрет*",
    "Палочки" : "Палочка*",
    "Отделы" : "Отдел*",
    "Книги" : "Книга*",
    "Классы" : "Класс*",
    "Зелья" : "Зелье*",
    "Заклятия" : "Заклятие*",
    "Декреты" : "Декрет*",
    "Фильмы" : "Гарри поттер и*(фильм)*"
}

for fn,mask in page_classes.items():
    with open(f'content/{fn}.txt','w',encoding='utf-8') as f:
        for ffn in glob(f'content/text/{mask}'):
            f.write(open(ffn).read())

Полученные файлы мы можем по-очереди загружать в модель Claude 2, и просить его написать вопросы к тексту, используя, например, такой промпт:

```
В приложенном файле содержится информация из вселенной Гарри Поттера.
Твоя задача - внимательно прочитать
текст и сформулировать как можно больше (но не менее 50) вопросов к 
этому тексту. Пожалуйста, запиши вопросы и ответы на них (содержащиеся 
в тексте) в формате JSON следующим образом:
[
    {"request":"вопрос 1","response":"ответ 1"},
    {"request":"вопрос 2","response":"ответ 2"},
    ...
].
Напиши только те вопросы, ответы на которые содержатся в тексте. 
Пожалуйста, формулируй вопросы и ответы как можно более подробным образом.
```

Все вопросы поместим в файл `data/pairs_for_train_claude.json`. В этом репозитории уже содержится файл с такими вопросами, посмотрим на него:

In [3]:
rr = json.load(open('data/pairs_for_train_claude.json'))
print(len(rr))

310


Объединим эти два файла в один, который затем используем для до-обучения модели:

In [5]:
d1 = json.load(open('data/pairs_for_train_ygpt.json'))
d2 = json.load(open('data/pairs_for_train_claude.json'))
with open('data/pairs_for_train.json', 'w', encoding='utf-8') as f:
    json.dump(d1 + d2, f, ensure_ascii=False)

print(f"Всего пар: {len(d1+d2)}")

Всего пар: 1030
